In [61]:
%pip install nbformat openpyxl -q


Note: you may need to restart the kernel to use updated packages.


In [62]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')


In [63]:
# Cargar datasets
files = ['datasets/rm_21_22.xlsx', 'datasets/rm_22_23.xlsx', 'datasets/rm_23_24.xlsx', 'datasets/rm_24_25.xlsx']
dfs = []
for f in files:
    df = pd.read_excel(f)
    df['periodo'] = f.split('_')[1] + '-' + f.split('_')[2].replace('.xlsx', '')
    dfs.append(df)

df = pd.concat(dfs, ignore_index=True)
df


,Player,Nation,Pos,Age,MP,Starts,Min,90s,Gls,Ast,...,G+A-PK,xG.1,xAG.1,xG+xAG,npxG.1,npxG+xAG.1,Matches,periodo,▲,▼
0,David Alaba,at AUT,DF,29.0,30,30,2642.0,29.4,2.0,3.0,...,0.17,0.06,0.07,0.13,0.06,0.13,Matches,21-22,NaN,NaN
1,Marco Asensio,es ESP,"FW,MF",25.0,31,19,1731.0,19.2,10.0,0.0,...,0.52,0.39,0.13,0.52,0.39,0.52,Matches,21-22,NaN,NaN
2,Gareth Bale,wls WAL,FW,32.0,5,4,280.0,3.1,1.0,0.0,...,0.32,0.39,0.03,0.42,0.39,0.42,Matches,21-22,NaN,NaN
3,Karim Benzema,fr FRA,FW,33.0,32,31,2593.0,28.8,27.0,12.0,...,1.11,0.83,0.28,1.10,0.52,0.80,Matches,21-22,NaN,NaN
4,Antonio Blanco,es ESP,MF,21.0,1,0,31.0,0.3,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,Matches,21-22,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,Federico Valverde,uy URU,"MF,DF",26.0,36,34,3032.0,33.7,6.0,4.0,...,0.30,0.08,0.09,0.17,0.08,0.17,Matches,24-25,NaN,NaN
148,Lucas Vázquez,es ESP,DF,33.0,32,25,2189.0,24.3,1.0,5.0,...,0.25,0.06,0.19,0.25,0.06,0.25,Matches,24-25,NaN,NaN
149,Daniel Yáñez,es ESP,FW,17.0,1,0,2.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,Matches,24-25,NaN,NaN
150,Squad Total,NaN,NaN,26.6,38,418,342.0,38.0,78.0,55.0,...,3.24,1.98,1.44,3.42,1.70,3.13,NaN,24-25,NaN,NaN


In [64]:
# Limpieza
numeric_cols = df.select_dtypes(include=[np.number]).columns
categorical_cols = df.select_dtypes(include=['object']).columns

for col in numeric_cols:
    df[col].fillna(df[col].median(), inplace=True)

for col in categorical_cols:
    if col != 'periodo':
        df[col].fillna(df[col].mode()[0], inplace=True)

df = df.drop_duplicates()
df.shape


(152, 37)

In [65]:
# Escalamiento
df_scaled = df.copy()
scaler = StandardScaler()
df_scaled[numeric_cols] = scaler.fit_transform(df_scaled[numeric_cols])
df_scaled.describe()


,Age,MP,Starts,Min,90s,Gls,Ast,G+A,G-PK,PK,...,Gls.1,Ast.1,G+A.1,G-PK.1,G+A-PK,xG.1,xAG.1,xG+xAG,npxG.1,npxG+xAG.1
count,1.520000e+02,1.520000e+02,1.520000e+02,1.520000e+02,1.520000e+02,1.520000e+02,1.520000e+02,1.520000e+02,1.520000e+02,1.520000e+02,...,1.520000e+02,1.520000e+02,1.520000e+02,152.000000,1.520000e+02,1.520000e+02,1.520000e+02,1.520000e+02,1.520000e+02,1.520000e+02
mean,-5.843279e-18,4.382459e-17,-3.505967e-17,-1.110223e-16,-5.843279e-17,-2.921640e-17,-5.843279e-18,1.168656e-17,3.505967e-17,2.045148e-17,...,-5.258951e-17,3.505967e-17,-2.921640e-17,0.000000,5.258951e-17,-6.135443e-17,-1.168656e-16,-3.798131e-17,-8.180591e-17,9.933574e-17
std,1.003306e+00,1.003306e+00,1.003306e+00,1.003306e+00,1.003306e+00,1.003306e+00,1.003306e+00,1.003306e+00,1.003306e+00,1.003306e+00,...,1.003306e+00,1.003306e+00,1.003306e+00,1.003306,1.003306e+00,1.003306e+00,1.003306e+00,1.003306e+00,1.003306e+00,1.003306e+00
min,-1.469682e+00,-1.200006e+00,-3.608719e-01,-1.207326e+00,-1.530092e+00,-3.746254e-01,-3.898682e-01,-3.846413e-01,-3.781782e-01,-2.765070e-01,...,-4.976954e-01,-5.873448e-01,-5.948483e-01,-0.505930,-6.030930e-01,-6.209660e-01,-6.513154e-01,-6.867265e-01,-6.348363e-01,-6.986945e-01
25%,-8.873264e-01,-1.077141e+00,-3.608719e-01,-8.326599e-01,-9.332705e-01,-3.746254e-01,-3.898682e-01,-3.421827e-01,-3.781782e-01,-2.765070e-01,...,-4.976954e-01,-5.873448e-01,-5.225331e-01,-0.505930,-5.270746e-01,-5.176982e-01,-4.814889e-01,-5.111329e-01,-5.203700e-01,-5.119292e-01
50%,-1.108519e-01,9.884574e-02,-2.296457e-01,-2.433988e-01,8.785454e-02,-3.025456e-01,-2.889142e-01,-2.997241e-01,-2.984513e-01,-2.765070e-01,...,-3.981891e-01,-2.934576e-01,-3.131993e-01,-0.396889,-3.070215e-01,-4.144303e-01,-2.759093e-01,-3.275577e-01,-4.059036e-01,-3.166745e-01
75%,8.597411e-01,9.764481e-01,-9.841961e-02,8.167464e-01,5.960857e-01,-2.124458e-01,-8.700636e-02,-1.298898e-01,-1.987927e-01,-2.765070e-01,...,-1.260196e-02,1.065556e-01,2.704311e-03,-0.008429,2.505868e-02,1.212718e-01,9.949672e-02,1.952324e-01,8.057827e-02,1.587282e-01
max,2.606809e+00,1.467905e+00,4.210172e+00,2.327657e+00,2.013538e+00,5.752160e+00,6.273092e+00,6.026606e+00,6.239154e+00,5.814662e+00,...,5.074660e+00,5.094475e+00,5.449187e+00,5.436822,5.670422e+00,4.516611e+00,4.926146e+00,4.868417e+00,4.344449e+00,4.785415e+00


In [79]:
# Variables dummy
df_dummies = df.copy()
cat_cols = [c for c in categorical_cols if c != 'periodo' and df[c].nunique() <= 10]

if cat_cols:
    df_dummies = pd.get_dummies(df_dummies, columns=cat_cols, drop_first=True, dtype=int)

numeric_cols_final = df_dummies.select_dtypes(include=[np.number]).columns
scaler_final = StandardScaler()
df_dummies[numeric_cols_final] = scaler_final.fit_transform(df_dummies[numeric_cols_final])

df_dummies


,Player,Nation,Age,MP,Starts,Min,90s,Gls,Ast,G+A,...,▲,▼,"Pos_DF,MF",Pos_FW,"Pos_FW,DF","Pos_FW,MF",Pos_GK,Pos_MF,"Pos_MF,DF","Pos_MF,FW"
0,David Alaba,at AUT,0.859741,0.906240,-0.032807,1.872810,1.211558,-0.230466,-0.087006,-0.172348,...,Andriy Lunin,Andriy Lunin,-0.081379,-0.389249,-0.11547,-0.250873,-0.411377,-0.443678,-0.202721,-0.202721
1,Marco Asensio,es ESP,0.083267,0.976448,-0.153097,0.810332,0.260373,0.346173,-0.389868,0.039945,...,Andriy Lunin,Andriy Lunin,-0.081379,-0.389249,-0.11547,3.986087,-0.411377,-0.443678,-0.202721,-0.202721
2,Gareth Bale,wls WAL,1.442097,-0.848965,-0.317130,-0.881935,-1.241007,-0.302546,-0.389868,-0.342183,...,Andriy Lunin,Andriy Lunin,-0.081379,2.569047,-0.11547,-0.250873,-0.411377,-0.443678,-0.202721,-0.202721
3,Karim Benzema,fr FRA,1.636216,1.046656,-0.021871,1.815662,1.155606,1.571530,0.821579,1.271244,...,Andriy Lunin,Andriy Lunin,-0.081379,2.569047,-0.11547,-0.250873,-0.411377,-0.443678,-0.202721,-0.202721
4,Antonio Blanco,es ESP,-0.693208,-1.129798,-0.360872,-1.172338,-1.502116,-0.374625,-0.389868,-0.384641,...,Andriy Lunin,Andriy Lunin,-0.081379,-0.389249,-0.11547,-0.250873,-0.411377,2.253886,-0.202721,-0.202721
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,Federico Valverde,uy URU,0.277385,1.327489,0.010936,2.327657,1.612548,0.057854,0.013948,0.039945,...,Andriy Lunin,Andriy Lunin,-0.081379,-0.389249,-0.11547,-0.250873,-0.411377,-0.443678,4.932883,-0.202721
148,Lucas Vázquez,es ESP,1.636216,1.046656,-0.087484,1.344486,0.735966,-0.302546,0.114902,-0.129890,...,Andriy Lunin,Andriy Lunin,-0.081379,-0.389249,-0.11547,-0.250873,-0.411377,-0.443678,-0.202721,-0.202721
149,Daniel Yáñez,es ESP,-1.469682,-1.129798,-0.360872,-1.206160,-1.530092,-0.374625,-0.389868,-0.384641,...,Andriy Lunin,Andriy Lunin,-0.081379,2.569047,-0.11547,-0.250873,-0.411377,-0.443678,-0.202721,-0.202721
150,Squad Total,es ESP,0.393856,1.467905,4.210172,-0.809626,2.013538,5.247601,5.162599,5.262351,...,Andriy Lunin,Andriy Lunin,-0.081379,-0.389249,-0.11547,-0.250873,-0.411377,-0.443678,-0.202721,-0.202721


In [67]:
# Top 10 Goleadores
top_goleadores = df.nlargest(10, 'Gls')[['Player', 'periodo', 'Gls', 'Ast', 'G+A', 'MP', 'Min']]
fig = px.bar(top_goleadores, x='Player', y='Gls', color='periodo',
            color_discrete_sequence=['#6366f1', '#8b5cf6', '#ec4899', '#f59e0b'])
fig.update_layout(title='Top 10 Goleadores', template='plotly_white', xaxis_tickangle=-45)
fig.show()
top_goleadores


,Player,periodo,Gls,Ast,G+A,MP,Min
112,Andriy Lunin,23-24,85.0,66.0,151.0,38,342.0
36,Squad Total,21-22,80.0,59.0,139.0,38,342.0
150,Squad Total,24-25,78.0,55.0,133.0,38,342.0
72,Andriy Lunin,22-23,73.0,56.0,129.0,38,342.0
151,Opponent Total,24-25,37.0,27.0,64.0,38,342.0
73,Andriy Lunin,22-23,35.0,21.0,56.0,38,342.0
37,Opponent Total,21-22,31.0,20.0,51.0,38,342.0
134,Kylian Mbappé,24-25,31.0,3.0,34.0,34,2907.0
3,Karim Benzema,21-22,27.0,12.0,39.0,32,2593.0
113,Andriy Lunin,23-24,25.0,17.0,42.0,38,342.0


In [68]:
# Top 10 Asistidores
top_asistidores = df.nlargest(10, 'Ast')[['Player', 'periodo', 'Ast', 'Gls', 'G+A', 'MP']]
fig = px.bar(top_asistidores, x='Player', y='Ast', color='periodo',
            color_discrete_sequence=['#6366f1', '#8b5cf6', '#ec4899', '#f59e0b'])
fig.update_layout(title='Top 10 Asistidores', template='plotly_white', xaxis_tickangle=-45)
fig.show()
top_asistidores


,Player,periodo,Ast,Gls,G+A,MP
112,Andriy Lunin,23-24,66.0,85.0,151.0,38
36,Squad Total,21-22,59.0,80.0,139.0,38
72,Andriy Lunin,22-23,56.0,73.0,129.0,38
150,Squad Total,24-25,55.0,78.0,133.0,38
151,Opponent Total,24-25,27.0,37.0,64.0,38
73,Andriy Lunin,22-23,21.0,35.0,56.0,38
37,Opponent Total,21-22,20.0,31.0,51.0,38
113,Andriy Lunin,23-24,17.0,25.0,42.0,38
3,Karim Benzema,21-22,12.0,27.0,39.0,32
17,Vinicius Júnior,21-22,10.0,17.0,27.0,35


In [69]:
# Goles vs Asistencias
fig = px.scatter(df, x='Gls', y='Ast', hover_data=['Player', 'periodo'], 
                size='G+A', color='periodo',
                color_discrete_sequence=['#6366f1', '#8b5cf6', '#ec4899', '#f59e0b'])
fig.update_layout(title='Goles vs Asistencias', template='plotly_white')
fig.show()


In [70]:
# Evolución de Goles por Periodo
goles_periodo = df.groupby('periodo')['Gls'].sum().sort_index()
fig = px.bar(x=goles_periodo.index, y=goles_periodo.values,
            color=goles_periodo.index,
            color_discrete_sequence=['#6366f1', '#8b5cf6', '#ec4899', '#f59e0b'])
fig.update_layout(title='Total de Goles por Periodo', xaxis_title='Periodo', 
                 yaxis_title='Goles', template='plotly_white', showlegend=False)
fig.show()


In [71]:
# Distribución de Goles, Asistencias y Partidos
fig = make_subplots(rows=1, cols=3, subplot_titles=['Goles', 'Asistencias', 'Partidos'])

fig.add_trace(go.Histogram(x=df['Gls'], marker_color='#6366f1', name='Gls'), row=1, col=1)
fig.add_trace(go.Histogram(x=df['Ast'], marker_color='#8b5cf6', name='Ast'), row=1, col=2)
fig.add_trace(go.Histogram(x=df['MP'], marker_color='#ec4899', name='MP'), row=1, col=3)

fig.update_layout(height=400, showlegend=False, title='Distribución de Métricas', template='plotly_white')
fig.show()


In [72]:
# Correlación entre métricas de rendimiento
metricas = ['Gls', 'Ast', 'G+A', 'MP', 'Min', 'xG', 'xAG', 'CrdY']
corr = df[metricas].corr()
fig = px.imshow(corr, text_auto='.2f', color_continuous_scale='RdBu_r', aspect='auto')
fig.update_layout(title='Correlación entre Métricas de Rendimiento', template='plotly_white', height=600)
fig.show()


In [87]:
 df_scaled.select_dtypes(include=['number'])

,Age,MP,Starts,Min,90s,Gls,Ast,G+A,G-PK,PK,...,Gls.1,Ast.1,G+A.1,G-PK.1,G+A-PK,xG.1,xAG.1,xG+xAG,npxG.1,npxG+xAG.1
0,0.859741,0.906240,-0.032807,1.872810,1.211558,-0.230466,-0.087006,-0.172348,-0.218724,-0.276507,...,-0.323559,-0.260803,-0.336036,-0.315108,-0.331027,-0.466064,-0.401045,-0.479207,-0.463137,-0.477972
1,0.083267,0.976448,-0.153097,0.810332,0.260373,0.346173,-0.389868,0.039945,0.419091,-0.276507,...,0.795887,-0.587345,0.196814,0.911607,0.229108,0.385896,-0.186527,0.143352,0.481210,0.184196
2,1.442097,-0.848965,-0.317130,-0.881935,-1.241007,-0.302546,-0.389868,-0.342183,-0.298451,-0.276507,...,0.298355,-0.587345,-0.107672,0.366400,-0.090969,0.385896,-0.544057,-0.016278,0.481210,0.014410
3,1.636216,1.046656,-0.021871,1.815662,1.155606,1.571530,0.821579,1.271244,1.216360,3.987311,...,1.840704,0.784129,1.460428,1.375033,1.173336,1.521842,0.349767,1.069210,0.853226,0.659599
4,-0.693208,-1.129798,-0.360872,-1.172338,-1.502116,-0.374625,-0.389868,-0.384641,-0.378178,-0.276507,...,-0.497695,-0.587345,-0.594848,-0.505930,-0.603093,-0.620966,-0.651315,-0.686726,-0.634836,-0.698695
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,0.277385,1.327489,0.010936,2.327657,1.612548,0.057854,0.013948,0.039945,0.100183,-0.276507,...,-0.049917,-0.195495,-0.138120,-0.015244,-0.122977,-0.414430,-0.329539,-0.415355,-0.405904,-0.410057
148,1.636216,1.046656,-0.087484,1.344486,0.735966,-0.302546,0.114902,-0.129890,-0.298451,-0.276507,...,-0.398189,0.098392,-0.214242,-0.396889,-0.202996,-0.466064,0.027991,-0.287650,-0.463137,-0.274228
149,-1.469682,-1.129798,-0.360872,-1.206160,-1.530092,-0.374625,-0.389868,-0.384641,-0.378178,-0.276507,...,-0.497695,-0.587345,-0.594848,-0.505930,-0.603093,-0.620966,-0.651315,-0.686726,-0.634836,-0.698695
150,0.393856,1.467905,4.210172,-0.809626,2.013538,5.247601,5.162599,5.262351,5.043250,5.814662,...,4.602005,4.147505,4.733647,4.373669,4.582159,4.490794,4.497111,4.772639,4.229983,4.615628


In [96]:
# Aplicar PCA con todas las componentes
df_pca = df_scaled.select_dtypes(include=['number'])

pca = PCA()
componentes = pca.fit_transform(df_numerico)

# Varianza explicada
varianza_df = pd.DataFrame({
    'Componente': [f'PC{i+1}' for i in range(len(pca.explained_variance_ratio_))],
    'Varianza_Explicada': pca.explained_variance_ratio_ * 100,
    'Varianza_Acumulada': np.cumsum(pca.explained_variance_ratio_) * 100
}).round(2)

varianza_df


,Componente,Varianza_Explicada,Varianza_Acumulada
0,PC1,74.81,74.81
1,PC2,8.07,82.88
2,PC3,4.02,86.91
3,PC4,2.90,89.80
4,PC5,2.64,92.44
5,PC6,2.02,94.46
6,PC7,1.52,95.99
7,PC8,1.36,97.34
8,PC9,0.76,98.10
9,PC10,0.59,98.70


In [94]:
# Gráfico de varianza explicada por componente
fig = go.Figure()
fig.add_trace(go.Bar(x=[f'PC{i+1}' for i in range(len(pca.explained_variance_ratio_))], 
                     y=pca.explained_variance_ratio_ * 100,
                     name='Varianza Explicada', marker_color='#6366f1'))
fig.add_trace(go.Scatter(x=[f'PC{i+1}' for i in range(len(pca.explained_variance_ratio_))], 
                         y=np.cumsum(pca.explained_variance_ratio_) * 100,
                         name='Varianza Acumulada', mode='lines+markers', 
                         marker_color='#ec4899', line=dict(width=3)))
fig.update_layout(title='Varianza Explicada por Componente Principal',
                 xaxis_title='Componente', yaxis_title='Varianza (%)',
                 template='plotly_white', height=500)
fig.show()


In [98]:
# Correlación entre Edad, Goles y Asistencias
corr_edad = df[['Age', 'Gls', 'Ast']].corr()
corr_edad


,Age,Gls,Ast
Age,1.000000,0.134707,0.156363
Gls,0.134707,1.000000,0.960968
Ast,0.156363,0.960968,1.000000


In [99]:
# Edad vs Goles
fig = px.scatter(df, x='Age', y='Gls', hover_data=['Player', 'periodo'],
                size='MP', color='periodo', trendline='ols',
                color_discrete_sequence=['#6366f1', '#8b5cf6', '#ec4899', '#f59e0b'])
fig.update_layout(title=f'Edad vs Goles (Correlación: {df["Age"].corr(df["Gls"]):.2f})',
                 xaxis_title='Edad', yaxis_title='Goles', template='plotly_white')
fig.show()


In [100]:
# Edad vs Asistencias
fig = px.scatter(df, x='Age', y='Ast', hover_data=['Player', 'periodo'],
                size='MP', color='periodo', trendline='ols',
                color_discrete_sequence=['#6366f1', '#8b5cf6', '#ec4899', '#f59e0b'])
fig.update_layout(title=f'Edad vs Asistencias (Correlación: {df["Age"].corr(df["Ast"]):.2f})',
                 xaxis_title='Edad', yaxis_title='Asistencias', template='plotly_white')
fig.show()


In [101]:
# Heatmap Edad-Goles-Asistencias
fig = px.imshow(corr_edad, text_auto='.3f', color_continuous_scale='RdBu_r', aspect='auto')
fig.update_layout(title='Correlación: Edad, Goles y Asistencias',
                 template='plotly_white', height=500, width=500)
fig.show()
